<a href="https://colab.research.google.com/github/MarlonGrandy/LSTMMusicGeneration/blob/main/LSTMMusicGenerationPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# import statements
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, stream, duration
import glob
import os
from itertools import chain
import copy
import numpy as np
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.layers import CuDNNLSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from numpy import argmax
from fractions import Fraction
import gc
import random
import keras.utils

In [35]:
def data_extractor(directory):
    """
    Summary:
    Function converts midi files to metadata and appends nested metadata lists into one large list
    composed of all the songs in the dataset.

    Parameters:
    directory: String representation of directory where midi files are located

    Returns:
    list: sequential midi file metadata
    """
    notes = []
    offsets = []
    durations = []
    for file in glob.glob(directory):
        print("File being parsed: " + file)
        mid = converter.parse(file)
        notes_to_parse = None
        prev_offset = 0

        try:
            s2 = instrument.partitionByInstrument(mid)
            notes_to_parse = s2.recurse()


        except:
            notes_to_parse = mid.flat.notes

        for i, element in enumerate(notes_to_parse):
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))

                durations.append(str(element.quarterLength))

                offset_dif = float(element.offset - prev_offset)

                offsets.append(round(offset_dif, 3))
                prev_offset = element.offset

            elif isinstance(element, chord.Chord):
                notes.append(".".join(str(n) for n in element.normalOrder))
                offset_dif = float(element.offset - prev_offset)

                durations.append(str(element.quarterLength))

                offsets.append(round(offset_dif, 3))
                prev_offset = element.offset

    return [notes, offsets, durations]

In [42]:
import music21 as m

song = m.converter.parse("./midi_files/chopin/chp_op31.mid")
# process the ties
song = song.stripTies()

# unfold repetitions
i = 0;
for a in song:
    if a.isStream:
        e = m.repeat.Expander(a)
        s2 = e.process()
        timing = s2.secondsMap
        song[i] = s2
    i += 1;

# todo: add note onsets

def getMusicProperties(x):
    s = '';
    t='';
    s = str(x.pitch) + ", " + str(x.duration.type) + ", " + str(x.duration.quarterLength);
    s += ", "
    if x.tie != None:
        t = x.tie.type;
    s += t + ", " + str(x.pitch.ps) + ", " + str(x.octave); # + str(x.seconds)  # x.seconds not always there  
    return s


print('pitch, duration_string, duration, tie, midi pitch, octave')
for a in song.recurse().notes:

    if (a.isNote):
        x = a;
        s = getMusicProperties(x);
        print(s);

    if (a.isChord):
        for x in a._notes:
            s = getMusicProperties(x);
            print(s);

print("Done.")

pitch, duration_string, duration, tie, midi pitch, octave
B-3, quarter, 1.0, , 58.0, 3
A3, quarter, 1.0, , 57.0, 3
B-3, quarter, 1.0, , 58.0, 3
C#4, quarter, 1.0, , 61.0, 4
F4, quarter, 1.0, , 65.0, 4
A3, quarter, 1.0, , 57.0, 3
B-3, quarter, 1.0, , 58.0, 3
C#4, quarter, 1.0, , 61.0, 4
F4, quarter, 1.0, , 65.0, 4
B-3, 16th, 0.25, , 58.0, 3
F#6, quarter, 1.0, , 90.0, 6
B-5, quarter, 1.0, , 82.0, 5
B-6, quarter, 1.0, , 94.0, 6
E-5, quarter, 1.0, , 75.0, 5
C6, quarter, 1.0, , 84.0, 6
E-6, quarter, 1.0, , 87.0, 6
C6, quarter, 1.0, , 84.0, 6
F5, quarter, 1.0, , 77.0, 5
F6, quarter, 1.0, , 89.0, 6
F#6, quarter, 1.0, , 90.0, 6
F#5, quarter, 1.0, , 78.0, 5
C6, quarter, 1.0, , 84.0, 6
C#6, quarter, 1.0, , 85.0, 6
F5, quarter, 1.0, , 77.0, 5
F6, quarter, 1.0, , 89.0, 6
A3, quarter, 1.0, , 57.0, 3
B-3, quarter, 1.0, , 58.0, 3
C#4, quarter, 1.0, , 61.0, 4
F4, quarter, 1.0, , 65.0, 4
A3, quarter, 1.0, , 57.0, 3
B-3, quarter, 1.0, , 58.0, 3
C#4, quarter, 1.0, , 61.0, 4
F4, quarter, 1.0, , 65.0, 4
G#

In [43]:
data = data_extractor("./midi_files/chopin/*.mid")
note_data = data[0]
offset_data = data[1]
duration_data = data[2]

unique_note_number = len(list(set(note_data)))
unique_notes = sorted(list(set(note_data)))
unique_offset_number = len(list(set(offset_data)))
unique_offsets = sorted(list(set(offset_data)))
unique_duration_number = len(list(set(duration_data)))
unique_durations = sorted(list(set(duration_data)))

File being parsed: ./midi_files/chopin/chpn_op23.mid
File being parsed: ./midi_files/chopin/chpn-p19.mid
File being parsed: ./midi_files/chopin/chpn_op7_2.mid
File being parsed: ./midi_files/chopin/chpn-p18.mid
File being parsed: ./midi_files/chopin/chpn-p24.mid
File being parsed: ./midi_files/chopin/chpn_op7_1.mid
File being parsed: ./midi_files/chopin/chpn-p23.mid
File being parsed: ./midi_files/chopin/chpn-p9.mid
File being parsed: ./midi_files/chopin/chpn-p8.mid
File being parsed: ./midi_files/chopin/chpn-p22.mid
File being parsed: ./midi_files/chopin/chpn-p20.mid
File being parsed: ./midi_files/chopin/chpn-p21.mid
File being parsed: ./midi_files/chopin/chp_op18.mid
File being parsed: ./midi_files/chopin/chpn_op35_4.mid
File being parsed: ./midi_files/chopin/chp_op31.mid
File being parsed: ./midi_files/chopin/chpn_op25_e4.mid
File being parsed: ./midi_files/chopin/chpn_op25_e1.mid


/Users/anguyen/miniforge3/envs/env_tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2002 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


File being parsed: ./midi_files/chopin/chpn_op33_4.mid


/Users/anguyen/miniforge3/envs/env_tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2003 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


File being parsed: ./midi_files/chopin/chpn_op53.mid
File being parsed: ./midi_files/chopin/chpn_op35_3.mid


...

File being parsed: ./midi_files/chopin/chpn-p6.mid


/Users/anguyen/miniforge3/envs/env_tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2002 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


File being parsed: ./midi_files/chopin/chpn-p7.mid
File being parsed: ./midi_files/chopin/chpn-p11.mid
File being parsed: ./midi_files/chopin/chpn-p13.mid
File being parsed: ./midi_files/chopin/chpn_op25_e11.mid


/Users/anguyen/miniforge3/envs/env_tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Et\xfcde Opus 25, No. 11'>; getting generic Instrument
  warnings.warn(
/Users/anguyen/miniforge3/envs/env_tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Sturmet\xfcde'>; getting generic Instrument
  warnings.warn(


File being parsed: ./midi_files/chopin/chpn-p5.mid
File being parsed: ./midi_files/chopin/chpn-p4.mid
File being parsed: ./midi_files/chopin/chpn-p12.mid
File being parsed: ./midi_files/chopin/chpn-p16.mid
File being parsed: ./midi_files/chopin/chpn_op27_2.mid
File being parsed: ./midi_files/chopin/chpn-p1.mid
File being parsed: ./midi_files/chopin/chpn-p17.mid
File being parsed: ./midi_files/chopin/chpn-p15.mid
File being parsed: ./midi_files/chopin/chpn-p3.mid
File being parsed: ./midi_files/chopin/chpn_op27_1.mid
File being parsed: ./midi_files/chopin/chpn-p2.mid
File being parsed: ./midi_files/chopin/chpn-p14.mid


In [ ]:
#6417368928

In [44]:
def one_hot_encode(vector, all_values):
    encoded_vectors = []
    int_to_index = dict((c, i) for i, c in enumerate(all_values))
    for i in vector:
        zero = [0] * (len(all_values) - 1)
        zero.insert(int_to_index[i], 1)
        encoded_vectors.append(zero)

    return encoded_vectors

In [45]:
from numpy import argmax


def one_hot_decode(vector, all_values):
    decoded_vector = []
    index_to_int = dict((i, c) for i, c in enumerate(all_values))
    decoded_vector.append(index_to_int[argmax(vector)])
    return decoded_vector

In [46]:
segment_length = 64


def make_segments(data_array, unique_values, seq_length=segment_length):
    input_seq = []
    output_seq = []

    processed_data = one_hot_encode(data_array, unique_values)

    for i in range(0, len(processed_data) - seq_length, 1):
        input_seq.append([processed_data[i : i + seq_length]])
        output_seq.append(processed_data[seq_length + i])

    del processed_data
    gc.collect()

    input_seq = np.reshape(
        input_seq, (len(input_seq), segment_length, len(unique_values))
    )
    output_seq = np.array(output_seq)

    return input_seq, output_seq

In [47]:
note_model_data = make_segments(data_array=note_data, unique_values=unique_notes)
offset_model_data = make_segments(data_array=offset_data, unique_values=unique_offsets)
duration_model_data = make_segments(
    data_array=duration_data, unique_values=unique_durations
)

In [48]:
X_train_note, X_test_note, y_train_note, y_test_note = train_test_split(
    note_model_data[0], note_model_data[1], test_size=0.2
)
X_train_off, X_test_off, y_train_off, y_test_off = train_test_split(
    offset_model_data[0], offset_model_data[1], test_size=0.2
)
X_train_dur, X_test_dur, y_train_dur, y_test_dur = train_test_split(
    duration_model_data[0], duration_model_data[1], test_size=0.2
)

In [49]:
del note_model_data
del offset_model_data
del duration_model_data
gc.collect()

0

In [50]:
def make_model(input_shape, output_shape):
    model = Sequential()
    model.add(CuDNNLSTM(512, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(0.25))
    model.add(Dense(128))
    model.add(Dense(output_shape, activation="softmax"))

    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(learning_rate=0.001),
        metrics=["accuracy"],
    )
    return model

In [52]:
model_notes = make_model(
    input_shape=(segment_length, unique_note_number), output_shape=unique_note_number
)
model_offsets = make_model(
    input_shape=(segment_length, unique_offset_number),
    output_shape=unique_offset_number,
)
model_durations = make_model(
    input_shape=(segment_length, unique_duration_number),
    output_shape=unique_duration_number,
)

In [53]:
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]

In [54]:
history_note = model_notes.fit(
    X_train_note, y_train_note, epochs=150, validation_split=0.2, callbacks=my_callbacks
)

Epoch 1/150


2022-12-05 20:50:10.887781: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-05 20:50:11.373753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - ETA: 0s - loss: 4.9591 - accuracy: 0.0248

2022-12-05 20:50:52.102532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - 46s 42ms/step - loss: 4.9591 - accuracy: 0.0248 - val_loss: 4.9309 - val_accuracy: 0.0284
Epoch 2/150
1080/1080 [==============================] - 43s 40ms/step - loss: 5.0844 - accuracy: 0.0244 - val_loss: 4.9473 - val_accuracy: 0.0267
Epoch 3/150
1080/1080 [==============================] - 43s 40ms/step - loss: 4.9412 - accuracy: 0.0250 - val_loss: 4.9224 - val_accuracy: 0.0284
Epoch 4/150
1080/1080 [==============================] - 43s 40ms/step - loss: 4.9221 - accuracy: 0.0263 - val_loss: 4.9227 - val_accuracy: 0.0284
Epoch 5/150
1080/1080 [==============================] - 43s 40ms/step - loss: 4.9153 - accuracy: 0.0268 - val_loss: 4.9128 - val_accuracy: 0.0284
Epoch 6/150
1080/1080 [==============================] - 43s 40ms/step - loss: 4.9125 - accuracy: 0.0267 - val_loss: 4.9206 - val_accuracy: 0.0284
Epoch 7/150
1080/1080 [==============================] - 43s 40ms/step - loss: 4.9105 - accuracy: 0.0274 - val_loss: 4.9187 - val_

In [55]:
history_offset = model_offsets.fit(
    X_train_off, y_train_off, epochs=150, validation_split=0.2, callbacks=my_callbacks
)

Epoch 1/150


2022-12-05 20:55:15.556140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - ETA: 0s - loss: 0.9986 - accuracy: 0.6377

2022-12-05 20:55:52.469607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - 42s 38ms/step - loss: 0.9986 - accuracy: 0.6377 - val_loss: 0.7999 - val_accuracy: 0.7121
Epoch 2/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.7412 - accuracy: 0.7376 - val_loss: 0.7299 - val_accuracy: 0.7473
Epoch 3/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.6592 - accuracy: 0.7708 - val_loss: 0.6285 - val_accuracy: 0.7858
Epoch 4/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.6015 - accuracy: 0.7926 - val_loss: 0.5883 - val_accuracy: 0.7938
Epoch 5/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.5618 - accuracy: 0.8064 - val_loss: 0.5554 - val_accuracy: 0.8162
Epoch 6/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.5280 - accuracy: 0.8200 - val_loss: 0.5540 - val_accuracy: 0.8174
Epoch 7/150
1080/1080 [==============================] - 41s 38ms/step - loss: 0.4974 - accuracy: 0.8310 - val_loss: 0.5356 - val_

In [56]:
history_duration = model_durations.fit(
    X_train_dur, y_train_dur, epochs=150, validation_split=0.2, callbacks=my_callbacks
)

Epoch 1/150


2022-12-05 21:02:07.931059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - ETA: 0s - loss: 1.0376 - accuracy: 0.6988

2022-12-05 21:02:45.270210: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1080/1080 [==============================] - 43s 39ms/step - loss: 1.0376 - accuracy: 0.6988 - val_loss: 0.9543 - val_accuracy: 0.7170
Epoch 2/150
1080/1080 [==============================] - 42s 39ms/step - loss: 0.9373 - accuracy: 0.7133 - val_loss: 0.8913 - val_accuracy: 0.7235
Epoch 3/150
1080/1080 [==============================] - 42s 39ms/step - loss: 0.8769 - accuracy: 0.7270 - val_loss: 0.8255 - val_accuracy: 0.7425
Epoch 4/150
1080/1080 [==============================] - 42s 39ms/step - loss: 0.8045 - accuracy: 0.7488 - val_loss: 0.7768 - val_accuracy: 0.7616
Epoch 5/150
1080/1080 [==============================] - 42s 39ms/step - loss: 0.7516 - accuracy: 0.7681 - val_loss: 0.7306 - val_accuracy: 0.7777
Epoch 6/150
1080/1080 [==============================] - 42s 39ms/step - loss: 0.6980 - accuracy: 0.7872 - val_loss: 0.7053 - val_accuracy: 0.7899
Epoch 7/150
1080/1080 [==============================] - 45s 42ms/step - loss: 0.6524 - accuracy: 0.7995 - val_loss: 0.6585 - val_

In [57]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype("float64")
    preds = np.exp(np.log(preds) / temperature)
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [58]:
def music_maker(seed_vec, model_type, number_unique, unique, diversity, num_notes=64):
    music = []
    arr = np.zeros((len(seed_vec) + num_notes, number_unique))
    for c, i in enumerate(seed_vec):
        arr[c] = i

    for i in range(0, num_notes, 1):
        d_arr = np.zeros(number_unique)
        pred = model_type.predict(
            np.reshape(
                arr[i : len(arr) - num_notes + i],
                (1, len(arr[i : len(arr) - num_notes + i]), number_unique),
            ),
            verbose=0,
        )[0]
        diverse = sample(pred, diversity)
        d_arr[diverse] = 1
        music.append(one_hot_decode(d_arr, all_values=unique)[0])

        arr[len(seed_vec) + i] = d_arr

    return music

In [1]:
randnum = random.randrange(0, len(X_test_note))
test_note = X_test_note[randnum]
test_offset = X_test_off[randnum]
test_dur = X_test_dur[randnum]
generated_music_note = music_maker(
    test_note, model_notes, unique_note_number, unique_notes, 0.7
)
generated_music_offset = music_maker(
    test_offset, model_offsets, unique_offset_number, unique_offsets, 0.7
)
generated_music_duration = music_maker(
    test_dur, model_durations, unique_duration_number, unique_durations, 0.2
)

NameError: name 'random' is not defined

In [71]:
seed_notes = []
seed_offsets = []
seed_durations = []
for n in test_note:
    seed_notes.append(one_hot_decode(n, unique_notes)[0])
for o in test_offset:
    seed_offsets.append(one_hot_decode(o, unique_offsets)[0])
for d in test_dur:
    seed_durations.append(one_hot_decode(d, unique_durations)[0])

In [72]:
def to_midi(notes, offsets, durations):
    """
    Summary:
    Takes midi metadata and converts it to a Music21 stream. The stream can then easily be converted into a midi file.

    Parameters:
    metadata: midi metadata (notes,chords,offsets)

    Returns:
    list: Music21 stream object
    """
    offset = offsets[0]
    s = stream.Stream()
    for i, ele in enumerate(notes):
        if ele[0].isalpha():
            n = note.Note(ele)
            try:
                n.quarterLength = float(durations[i])
            except:
                n.quarterLength = Fraction(durations[i])

            s.insert(offset, n)

            offset += offsets[i]
        else:
            chords = list(map(int, ele.split(".")))
            c = chord.Chord(chords)

            try:
                c.quarterLength = float(durations[i])
            except:
                c.quarterLength = Fraction(durations[i])

            s.insert(offset, c)

            offset += offsets[i]
    return s

In [73]:
to_midi(generated_music_note, generated_music_offset, generated_music_duration).write(
    "midi", "generated_classical.mid"
)
to_midi(seed_notes, seed_offsets, seed_durations).write("midi", "seed_classical.mid")

'seed_classical.mid'